In [32]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# 1. Read files including `Answer`, `Choice`, `Question`

In [18]:
# Read Files to List
doc_num = 91
question_doc = []
choice_doc = []
ans_doc = []

for i in range(1, doc_num + 1):
    try:
        with open(f'Data/Question/{i}.txt', encoding='utf-8') as f:
            raw_q = f.read()
    except:
        with open(f'Data/Question/{i}.txt', encoding='big5') as f:
            raw_q = f.read()
    try:
        with open(f'Data/Choice/{i}.txt', encoding='utf-8') as f:
            raw_ch = f.read()
    except:
        with open(f'Data/Choice/{i}.txt', encoding='big5') as f:
            raw_ch = f.read()
    try:
        with open(f'Data/Answer/{i}.txt', encoding='utf-8') as f:
            raw_a = f.read()
    except:
        with open(f'Data/Answer/{i}.txt', encoding='big5') as f:
            raw_a = f.read()
    question_doc.append(raw_q)
    choice_doc.append(raw_ch.split('\n'))
    ans_doc.append(list(raw_a))

In [19]:
choice_cols = ['A', 'B', 'C', 'D', 'E', 'F']
choice_df = pd.DataFrame(choice_doc, columns=choice_cols, index=[i + 1 for i in range(len(choice_doc))])
for col in choice_cols:
    choice_df[col] = choice_df[col].str.replace(r'\([A-F]\)', '')

ans_cols = [1, 2, 3, 4, 5]
ans_df = pd.DataFrame(ans_doc, columns=ans_cols, index=[i + 1 for i in range(len(choice_doc))])

display(choice_df.head())
display(ans_df.head())

,A,B,C,D,E,F
1,Nothing is made up.,"History is nonfiction, too.",But your trip through space would be fiction.,You could write a story in which you fly to t...,But fiction isn’t always different from the w...,None
2,"But when I pulled into the driveway, there wa...",Hours later I called my daughter and asked if...,"One time, I even shouted at Derek when he unp...",Derek became part of our life and seemed to f...,"Without any experience in raising pets, my hu...",None
3,Another genre commonly found in Chinese brush...,"However, the subject matters later expanded b...","As a result, they have obtained more natural ...",Its growth has inevitably reflected the chang...,It then gradually developed into two separate...,
4,She got to look at the fine old houses as wel...,Architects make drawings and careful plans of...,The word for what she does is rehabilitation.,Then she went on to study architecture.,That is precisely what Carrie does.,None
5,There are definitely two sides to this issue.,Scores decreased as the family size increased...,An intelligence test was administered to over...,"On the other side are Rutherford and Sewell, ...","Since then, the theory has been elaborated an...",None


,1,2,3,4,5
1,E,D,C,A,B
2,E,D,C,A,B
3,D,B,E,A,C
4,C,E,A,D,B
5,C,B,E,A,D


# 2. BERT (NSP)
code ref: https://towardsdatascience.com/bert-for-next-sentence-prediction-466b67f8226f

In [20]:
# pip3 install transformers
# pip3 install torch

from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
count_NotNextSentence = 0
NotNextSentence = []

for i in range(len(question_doc)): # each document
  question_sentence = re.split('\(1\)|\(2\)|\(3\)|\(4\)|\(5\)', question_doc[i])
  for j in range(5): # each question
      for k in range(6): # each option
        if (choice_df.iloc[i][choice_cols[k]] != 'None'):
          inputs = tokenizer(question_sentence[j], choice_df.iloc[i][choice_cols[k]], return_tensors='pt')
          outputs = model(**inputs)
          if (torch.argmax(outputs.logits).item() == 1): # NotNextSentence
            count_NotNextSentence += 1
            NotNextSentence.append([i+1, j+1, choice_cols[k]]) # Doc i+1, Question j+1, Choice choice_cols[k], is not the answer
  print("=== Finish Doc", i+1, "===")

=== Finish Doc 1 ===
=== Finish Doc 2 ===
=== Finish Doc 3 ===
=== Finish Doc 4 ===
=== Finish Doc 5 ===
=== Finish Doc 6 ===
=== Finish Doc 7 ===
=== Finish Doc 8 ===
=== Finish Doc 9 ===
=== Finish Doc 10 ===
=== Finish Doc 11 ===
=== Finish Doc 12 ===
=== Finish Doc 13 ===
=== Finish Doc 14 ===
=== Finish Doc 15 ===
=== Finish Doc 16 ===
=== Finish Doc 17 ===
=== Finish Doc 18 ===
=== Finish Doc 19 ===
=== Finish Doc 20 ===
=== Finish Doc 21 ===
=== Finish Doc 22 ===
=== Finish Doc 23 ===
=== Finish Doc 24 ===
=== Finish Doc 25 ===
=== Finish Doc 26 ===
=== Finish Doc 27 ===
=== Finish Doc 28 ===
=== Finish Doc 29 ===
=== Finish Doc 30 ===
=== Finish Doc 31 ===
=== Finish Doc 32 ===
=== Finish Doc 33 ===
=== Finish Doc 34 ===
=== Finish Doc 35 ===
=== Finish Doc 36 ===
=== Finish Doc 37 ===
=== Finish Doc 38 ===
=== Finish Doc 39 ===
=== Finish Doc 40 ===
=== Finish Doc 41 ===
=== Finish Doc 42 ===
=== Finish Doc 43 ===
=== Finish Doc 44 ===
=== Finish Doc 45 ===
=== Finish Doc 46 =

In [64]:
# In all situations (91*5*5 = 2375), only `count_NotNextSentence` situations are defined as `NotNextSentence`
print(count_NotNextSentence)

195


In [65]:
print(NotNextSentence)

[[3, 4, 'D'], [4, 1, 'D'], [4, 2, 'C'], [4, 2, 'D'], [4, 2, 'E'], [4, 3, 'B'], [5, 3, 'A'], [6, 1, 'B'], [6, 4, 'B'], [8, 5, 'D'], [10, 1, 'A'], [10, 1, 'C'], [10, 1, 'D'], [10, 3, 'C'], [10, 5, 'A'], [13, 1, 'C'], [13, 2, 'C'], [14, 1, 'C'], [14, 2, 'C'], [14, 3, 'C'], [17, 5, 'F'], [18, 5, 'D'], [19, 1, 'B'], [19, 1, 'C'], [19, 2, 'C'], [19, 3, 'C'], [19, 4, 'B'], [20, 2, 'C'], [20, 3, 'C'], [20, 5, 'C'], [22, 1, 'A'], [22, 1, 'B'], [22, 1, 'D'], [22, 1, 'F'], [22, 2, 'B'], [22, 2, 'F'], [22, 3, 'B'], [22, 3, 'E'], [22, 3, 'F'], [22, 4, 'A'], [22, 4, 'D'], [22, 4, 'E'], [22, 5, 'A'], [22, 5, 'C'], [22, 5, 'E'], [23, 1, 'B'], [23, 5, 'A'], [23, 5, 'B'], [27, 4, 'C'], [31, 1, 'B'], [31, 1, 'C'], [31, 3, 'A'], [31, 4, 'A'], [31, 5, 'A'], [33, 1, 'A'], [33, 2, 'A'], [33, 5, 'A'], [33, 5, 'B'], [34, 2, 'A'], [34, 5, 'C'], [38, 1, 'B'], [38, 3, 'B'], [38, 5, 'B'], [41, 2, 'B'], [41, 4, 'B'], [43, 1, 'B'], [43, 4, 'B'], [43, 4, 'F'], [44, 1, 'B'], [44, 1, 'C'], [44, 5, 'A'], [45, 1, 'B'], [

In [76]:
for i in range(len(NotNextSentence)):
  if (ans_df.iloc[NotNextSentence[i][0]-1][NotNextSentence[i][1]] == NotNextSentence[i][2]):
    print("Wrong answer!!", NotNextSentence[i])

Wrong answer!! [4, 2, 'E']
Wrong answer!! [47, 4, 'A']
Wrong answer!! [52, 3, 'F']
Wrong answer!! [53, 4, 'B']
Wrong answer!! [63, 5, 'C']
